In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout

In [2]:
df = pd.read_csv('Dataset/test.csv')

In [3]:
df1 = pd.read_csv('Dataset/train.csv')

In [4]:
targets = df1[['pet_id','breed_category','pet_category']]

In [5]:
df1 = df1.drop('breed_category', axis = True)

In [6]:
df1= df1.drop('pet_category', axis = True)

In [7]:
test = pd.concat([df1,df])

In [8]:
test['issue_date'] = pd.to_datetime(test['issue_date'])

In [9]:
test['listing_date'] = pd.to_datetime(test['listing_date'])

In [10]:
test['days'] =  (test['listing_date'] - test['issue_date']).dt.days

In [11]:
test['condition'].fillna(1.0, inplace=True)

In [12]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [13]:
test = encode_and_bind(test, 'color_type')

In [14]:
test.drop('color_type',axis = 1, inplace = True)

In [15]:
test['height(m)']=test['height(cm)'].apply(lambda x:x*(.01))

In [16]:
train_df=test[:18834]

In [17]:
test_df= test[18834:]

In [18]:
train_df = pd.concat([train_df,targets], axis =1)

In [19]:
train_df.drop('height(cm)', axis = 1, inplace = True)
test_df.drop('height(cm)', axis = 1, inplace = True)

C:\Users\Ashish Bisht\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
train_df.drop('issue_date', axis = 1, inplace = True)
train_df.drop('listing_date', axis = 1, inplace=True)
test_df.drop('issue_date', axis = 1, inplace = True)
test_df.drop('listing_date', axis = 1, inplace=True)

In [21]:
train_df=train_df.drop(df.columns[0], axis=1)

In [22]:
train_df

,condition,length(m),X1,X2,days,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,...,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),breed_category,pet_category
0,2.0,0.80,13,9,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0778,0.0,1
1,1.0,0.72,13,9,1862,0,0,0,0,0,...,0,0,0,0,1,0,0,0.1419,0.0,2
2,1.0,0.15,15,4,752,0,0,0,0,0,...,0,0,0,0,0,0,0,0.4090,2.0,4
3,1.0,0.62,0,1,755,0,0,0,0,0,...,0,0,0,0,1,0,0,0.1782,0.0,2
4,2.0,0.50,18,4,52,0,0,1,0,0,...,0,0,0,0,0,0,0,0.1106,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,2.0,0.44,0,1,407,0,0,0,0,0,...,0,0,0,1,0,0,0,0.2736,0.0,2
18830,1.0,0.73,15,4,386,0,0,0,0,0,...,0,0,0,0,0,0,0,0.1425,2.0,4
18831,0.0,0.99,13,9,2954,0,0,0,0,0,...,0,0,0,0,0,0,0,0.2813,1.0,1
18832,0.0,0.55,13,9,272,0,0,0,0,0,...,0,0,0,0,0,0,0,0.4482,1.0,2


In [23]:
init_train_df = train_df.drop('pet_category', axis = 1)

In [24]:
init_train_df

,condition,length(m),X1,X2,days,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,...,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),breed_category
0,2.0,0.80,13,9,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0778,0.0
1,1.0,0.72,13,9,1862,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1419,0.0
2,1.0,0.15,15,4,752,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.4090,2.0
3,1.0,0.62,0,1,755,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1782,0.0
4,2.0,0.50,18,4,52,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0.1106,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,2.0,0.44,0,1,407,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.2736,0.0
18830,1.0,0.73,15,4,386,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.1425,2.0
18831,0.0,0.99,13,9,2954,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.2813,1.0
18832,0.0,0.55,13,9,272,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0.4482,1.0


In [25]:
X1_train = init_train_df.drop('breed_category', axis = 1)
Y1_test = init_train_df['breed_category']

In [26]:
X1_train.shape

(18834, 62)

In [27]:
Y1_test.nunique()

3

In [30]:
def baseline_model():

    model = Sequential()
    model.add(Dense(1024, input_dim=62, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [31]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose=1)
estimator.fit(X1_train,Y1_test)

Epoch 1/100
3767/3767 [==============================] - 19s 5ms/step - loss: 1.0810 - accuracy: 0.4632
Epoch 2/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.9110 - accuracy: 0.4748
Epoch 3/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.9057 - accuracy: 0.4748
Epoch 4/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8840 - accuracy: 0.4818
Epoch 5/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8733 - accuracy: 0.4918
Epoch 6/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8540 - accuracy: 0.4961
Epoch 7/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8409 - accuracy: 0.5046
Epoch 8/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8162 - accuracy: 0.5152
Epoch 9/100
3767/3767 [==============================] - 18s 5ms/step - loss: 0.8170 - accuracy: 0.5178
Epoch 10/100
3767/3767 [==============================] - 18s 5m

3767/3767 [==============================] - 14s 4ms/step - loss: 0.9132 - accuracy: 0.4779
Epoch 80/100
3767/3767 [==============================] - 17s 4ms/step - loss: 0.9133 - accuracy: 0.4779
Epoch 81/100
3767/3767 [==============================] - 16s 4ms/step - loss: 0.9133 - accuracy: 0.4779
Epoch 82/100
3767/3767 [==============================] - 15s 4ms/step - loss: 0.9132 - accuracy: 0.4779
Epoch 83/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.9132 - accuracy: 0.4779
Epoch 84/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.9199 - accuracy: 0.4779
Epoch 85/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.9135 - accuracy: 0.4779
Epoch 86/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.9132 - accuracy: 0.4779
Epoch 87/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.9133 - accuracy: 0.4779
Epoch 88/100
3767/3767 [==============================] - 14s 4ms/st

In [32]:
test1 = test_df.drop('pet_id', axis =1)

In [33]:
y1_result =estimator.predict(test1)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
1615/1615 [==============================] - 2s 1ms/step


In [34]:
sec_train_df = train_df.drop('breed_category', axis = 1)

In [35]:
X2_train = sec_train_df.drop('pet_category', axis = 1)
Y2_test = sec_train_df['pet_category']

In [36]:
X2_train.shape

(18834, 62)

In [37]:
def baseline_model():

    model = Sequential()
    model.add(Dense(1024, input_dim=62, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [38]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose=1)
estimator.fit(X2_train,Y2_test)

Epoch 1/100
3767/3767 [==============================] - 13s 3ms/step - loss: 1.1686 - accuracy: 0.5439
Epoch 2/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.8083 - accuracy: 0.6595
Epoch 3/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.7476 - accuracy: 0.7068
Epoch 4/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.7155 - accuracy: 0.7261
Epoch 5/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.6930 - accuracy: 0.7381
Epoch 6/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.7135 - accuracy: 0.7117
Epoch 7/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.7055 - accuracy: 0.7196
Epoch 8/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.6772 - accuracy: 0.7421
Epoch 9/100
3767/3767 [==============================] - 13s 3ms/step - loss: 0.6427 - accuracy: 0.7569
Epoch 10/100
3767/3767 [==============================] - 13s 3m

3767/3767 [==============================] - 14s 4ms/step - loss: 0.5523 - accuracy: 0.7935
Epoch 80/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5476 - accuracy: 0.7943
Epoch 81/100
3767/3767 [==============================] - 13s 4ms/step - loss: 0.5441 - accuracy: 0.7983
Epoch 82/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5302 - accuracy: 0.8003
Epoch 83/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5557 - accuracy: 0.7908
Epoch 84/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5521 - accuracy: 0.7949
Epoch 85/100
3767/3767 [==============================] - ETA: 0s - loss: 0.5264 - accuracy: 0.80 - 14s 4ms/step - loss: 0.5266 - accuracy: 0.8043
Epoch 86/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5625 - accuracy: 0.7944
Epoch 87/100
3767/3767 [==============================] - 14s 4ms/step - loss: 0.5485 - accuracy: 0.7987
Epoch 88/100
3767/3767 [==

In [39]:
test2 = test_df.drop('pet_id', axis =1)

In [40]:
y2_result = estimator.predict(test2)

1615/1615 [==============================] - 2s 1ms/step


In [41]:
y2_result

array([2, 1, 2, ..., 2, 2, 2], dtype=int64)

In [42]:
results_df = pd.DataFrame(data={'pet_id':test_df['pet_id'], 'breed_category':y1_result,'pet_category':y2_result})
results_df.to_csv('submission_neuralnet1.csv', index=False)